data의 text를 통해서 disaster관련 문장이면 target 1 아니면 0

initial method

text -> tokenized 

using pre trained word embedding (Word2Vec)

GRU (bidirectional)

result

input data : text

output data : target(digit 0,1)

model : architecture (GRU) // params : (embedding,GRU의 params)

cost : binary_cross_entropy

In [71]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from pandas import DataFrame as df
import nltk
from matplotlib import pyplot as plt

In [3]:
import os
os.getcwd()

'C:\\Users\\IDSL\\Desktop\\파이썬공부\\-Kaggle-Real-or-Not-NLP-with-Disaster-Tweets-day2\\-Kaggle-Real-or-Not-NLP-with-Disaster-Tweets-day2'

In [100]:
data=pd.read_csv('./train.csv',header=0,index_col=0)
data.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [127]:
# 1. text data pre processing
# cleanse, tokenize, encoding, vectorize

# 1.1. cleanse
import re
# 이전엔 그냥 str의 replace 활용했는데 #는 못바꾸넹..
# 대문자 -> 소문자
# 특수문자 제거 (구두점, 쉼표 제외)
# http 부분 제거 -> split으로 한 다음에 http 가 있다면 그 부분을 지우고 다시 합치자
def cleanse(sentence):
    result=sentence.lower()
    result=re.sub('[^a-z ,.]','',result)
    result=[i for i in result.split() if 'http' not in i]
    Result=''
    for i in result:
        Result=Result+i+' '
    return Result
data['post_text']=data['text'].apply(lambda i : cleanse(i))
print(data.post_text[:30])

id
1     our deeds are the reason of this earthquake ma...
4               forest fire near la ronge sask. canada 
5     all residents asked to shelter in place are be...
6     , people receive wildfires evacuation orders i...
7     just got sent this photo from ruby alaska as s...
8     rockyfire update california hwy. closed in bot...
10    flood disaster heavy rain causes flash floodin...
13    im on top of the hill and i can see a fire in ...
14    theres an emergency evacuation happening now i...
15    im afraid that the tornado is coming to our ar...
16         three people died from the heat wave so far 
17    haha south tampa is getting flooded hah wait a...
18    raining flooding florida tampabay tampa or day...
19               flood in bago myanmar we arrived bago 
20    damage to school bus on in multi car crash bre...
23                                        whats up man 
24                                       i love fruits 
25                                    summer 

In [139]:
# # of data
len(data) #7613
# train_dev split
from sklearn.model_selection import train_test_split
train_data,val_data=train_test_split(data,test_size=613,shuffle=True) # train data 7000, val data 613

In [81]:
# nltk로 tokenize
# 1.2. tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# train_text_tokenized=word_tokenize(train_data.post_text) #-> error 발생 -> 애초에 word_tokenize는 str을 위한 것

# http가 들어간 sentence들이 있어서 그 부분은 제거해주자.
def word_tokenize_1(sentence):
    j=word_tokenize(sentence)
    result=[i for i in j if 'http' not in i] 
    return result
sentence_tokenized=train_data['post_text'].apply(lambda i : word_tokenize_1(i))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\IDSL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [82]:
print(sentence_tokenized.head()) # http가 들어있는 부분들이 있는데 이는 제거하자.-> 제거 完

3037    [usgs, eq, m, ., km, ssw, of, anza, california...
5822    [chinas, stock, market, crash, are, there, gem...
6985    [the, sharper, image, viper, hardside, twister...
5893    [socialwots, globiinclusion, rt, nrcmiddleeast...
3690    [i, liked, a, youtube, video, from, vgbootcamp...
Name: post_text, dtype: object


In [83]:
# 1.3 word embedding -> using pre trained word2vec -> 너무 커서 일단은 pretrained된 것을 활용 O -> 구글
from gensim.models import Word2Vec,KeyedVectors
# KeyedVectors는 학습이 안된다...
# word2vec=KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary=True)

#  일단 training data만으로 word2vec을 학습시켜서 진행을 하자.(계산의 편의를 위해)
word2vec_toy=Word2Vec(sentence_tokenized,size=300,min_count=3,window=3)

In [92]:
# 1.3.1 wordembedding -> pytorch & word encoding 
# 직접 일일히 짜지 말고 torchtext를 활용하자
import torchtext
word2vec_toy_vocab=tuple(word2vec_toy.wv.vocab.keys())
word2index={word:index+2 for index,word in enumerate(word2vec_toy_vocab)} # 0 for padding 1 for unk
index2word={index+2:word for index,word in enumerate(word2vec_toy_vocab)} # 0 for padding 1 for unk


# preprocessing with pytorch

In [158]:
# 1 preprocessing
# torchtext를 활용하면 한방에(tokenize,wordembedding,padding ...)
# SOS,EOS,padding,fix_len : 40 (애초에 twitter는 글자수 제한이 280)

# Field 정의
Text=torchtext.data.Field(sequential=True,use_vocab=True, init_token='<SOS>',eos_token='<EOS>',tokenize=word_tokenize,batch_first=True,lower=True,fix_length=40)
Target=torchtext.data.Field(sequential=False,use_vocab=False,batch_first=True,is_target=True)
Id=torchtext.data.Field(sequential=False,use_vocab=False,batch_first=True)

# Data 정의
# 영어와 ., 만을 남긴 csv 형태로 넘겨주자
# 또한 http부분을 제거
train_data[['post_text','target']].to_csv('train_data.csv',index=True)
train_data_1=torchtext.data.TabularDataset('./train_data.csv',format='csv',fields=[('id',Id),('text',Text),('target',Target)],
                                           skip_header=True)


In [159]:
# vars 객체의 내부 변수가 저장된 딕셔너리 반환
vars(train_data_1[10])

{'id': '2454',
 'text': ['cyclist',
  'who',
  'collided',
  'with',
  'runner',
  'on',
  'roanoke',
  'greenway',
  'wins',
  'civil',
  'verdict',
  'via',
  'roanoketimes'],
 'target': '0'}

In [223]:
# vocabulary 만들기
Text.build_vocab(train_data_1,min_freq=3)
# 단어 집합의 크기
# print(len(Text.vocab))
# 단어 집합의 내부 단어들 확인하기
# print(Text.vocab.stoi)
# 싹 다 vectorized도 됬다.

# 근데 사전 훈련된 Word2Vec을 초기 임베딩으로 활용하기, Word2Vec 토큰화 된 녀석을 받는다
#  torchtext.vocab.Vectors(name<-name of files)
# https://discuss.pytorch.org/t/aligning-torchtext-vocab-index-to-loaded-embedding-pre-trained-weights/20878/2

word2vec_toy=Word2Vec(train_data['post_text'].apply(lambda i : word_tokenize(i))
                      ,size=300,min_count=3,window=3)
# Word2Vec.wv.vocab.keys()
word2vec_toy.wv.save_word2vec_format('./wv',binary=False)
# # Text.build_vocab(train_data_1,min_freq=3,vectors=word2vec_toy)
# # print(Text.vocab.stoi)


In [225]:
torchtext.vocab.Vectors('wv',cache='./')

100%|███████████████████████████████████████████████████████████████████████████▊| 4081/4092 [00:00<00:00, 5800.87it/s]


In [179]:
torchtext.vocab.Vectors('word2vec.model',cache='./')

  0%|                                                                                           | 0/28 [00:00<?, ?it/s]Skipping token b'28' with 1-dimensional vector [b'300']; likely a header



ValueError: could not convert string to float: 

In [164]:
# Iterator로 data 불러오기
batch_size=128
train_loader=torchtext.data.Iterator(train_data_1,batch_size=batch_size)
print(next(iter(train_loader)).text)

tensor([[  2,   0,   0,  ...,   1,   1,   1],
        [  2, 537, 437,  ...,   1,   1,   1],
        [  2, 129,   7,  ...,   1,   1,   1],
        ...,
        [  2,   0,   0,  ...,   1,   1,   1],
        [  2,  45, 114,  ...,   1,   1,   1],
        [  2,   0,   0,  ...,   1,   1,   1]])
